# Exploring Shastry-Sutterland model with RBM variational wave function and other models

In [82]:
import netket as nk
import numpy as np
import time
import json
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import jax
import flax
import optax
from sys import version as pyvers

print("Python version: {}".format(pyvers))
print("NetKet version: {}".format(nk.__version__))
print("NumPy version: {}".format(np.__version__))

Python version: 3.8.10 (default, Nov 26 2021, 20:14:08) 
[GCC 9.3.0]
NetKet version: 3.3.2.post2
NumPy version: 1.20.3


### Setup relevant parameters and settings of the simulation 

In [83]:
"""lattice"""
SITES    = 16            # 4, 8, 16, 20 ... number of vertices in a tile determines the tile shape 
JEXCH1   = .2            # nn interaction
JEXCH2   = 1             # nnn interaction
H_Z      = 0             # magnetic field
USE_PBC = True
TOTAL_SZ = None          # 0, None ... restriction of Hilbert space
#USE_MSR = True          # Should we use a Marshall sign rule? In this notebook, we use both.

"""machine learning"""
MACHINE  = "GCNN"         # RBM, RBMSymm, RBMSymm_transl, RBMModPhase, GCNN, Jastrow, myRBM
DTYPE    = np.complex128 # data-type of weights in neural network
ALPHA    = 2            # N_hidden / N_visible
ETA      = .005          # learning rate (0.01 usually works)
SAMPLER  = 'local'       # 'local' = MetropolisLocal, 'exchange' = MetropolisExchange
SAMPLES  = 700          # number of Monte Carlo samples
NUM_ITER = 20           # number of convergence iterations
N_LAYERS = 2 #2             # number of layers (in case of G-CNN)
FEATURES = (16,16) #16 #(8,4)         # dimensions of layers (in case of G-CNN)

OUT_NAME = "SS_"+str(SITES)+"j1="+str(JEXCH1) # output file name

### Lattice definition
Basic structure of tiled lattices is implemented in `lattice_and_ops.py` file. Class `Lattice` implements relative positional functions, 
- e.g. *`rt(node)` returns the index of the right neighbour of a site with index `node`*

The `for` loop constructs full Shastry-Sutherland lattice with PBC using these auxiliary positional functions.

In [84]:
from lattice_and_ops import Lattice
lattice = Lattice(SITES)

if not USE_PBC and SITES != 16:
    raise Exception("Non-PBC are implemented only for 4x4 lattice!!!")

# Construction of custom graph according to tiled lattice structure defined in the Lattice class.
edge_colors = []
for node in range(SITES):
    if USE_PBC or not node in [3,7,11,15]:
        edge_colors.append([node,lattice.rt(node), 1])  # horizontal connections
    if USE_PBC or not node in [12,13,14,15]:
        edge_colors.append([node,lattice.bot(node), 1]) # vertical connections
    row, column = lattice.position(node)
    
    SS_color = 3 if not USE_PBC and node in [3,7,4,12,13,14,15] else 2
    if column%2 == 0:
        if row%2 == 0:
            edge_colors.append([node,lattice.lrt(node),SS_color]) # diagonal bond
        else:
            edge_colors.append([node,lattice.llft(node),SS_color]) # diagonal bond

g = nk.graph.Graph(edges=edge_colors) #,n_nodes=3)
N = g.n_nodes

hilbert = nk.hilbert.Spin(s=.5, N=g.n_nodes, total_sz=TOTAL_SZ)

In [85]:
i = 0
while sum(lattice.position(i))%2 == 0: # ensure that the plaquette is 'empty' (without J_2 bond inside)
    i += 1

In [86]:
lattice.position(1)

(0, 1)

In [87]:
g.edges(filter_color=2)

[(0, 5), (2, 7), (4, 11), (6, 9), (8, 13), (10, 15), (3, 12), (1, 14)]

### Characters of the symmetries
In case of G-CNN, we need to specify the characters of the symmetry transformations.
- DS phase anti-symmetric wrt permutations with negative sign and symmetric wer permutaions with postive sign
- AF phase is always symmetric for all permutations  

In [88]:

print("There are", len(g.automorphisms()), "full symmetries.")
# deciding point between DS and AF phase is set to 0.5
if JEXCH1 < 0.5:
    # DS phase is partly anti-symmetric
    characters = []
    from lattice_and_ops import permutation_sign
    for perm in g.automorphisms():
        # print(perm, "with sign", permutation_sign(np.asarray(perm)))
        characters.append(permutation_sign(np.asarray(perm)))
    characters_dimer_1 = np.asarray(characters,dtype=complex)
    characters_dimer_2 = characters_dimer_1
else:
    # AF phase if fully symmetric
    characters_dimer_1 = np.ones((len(g.automorphisms()),), dtype=complex)
    characters_dimer_2 = characters_dimer_1

There are 64 full symmetries.


### Translations

If we want to include only translations, we have to exclude some symmetries from `g.automorphisms()`.


⚠️ TODO ⚠️ <span style="color:red"> This part is not fully automated yet. Translations are currently picked by hand from the group of all automorphisms. </span>

In [89]:
if MACHINE == "RBMSymm_transl" and not (N in [4,16]):
    raise NotImplementedError("Extraction of translations from the group of automorphisms is not implemented yet.")
translations = []
for perm in g.automorphisms():
    aperm = np.asarray(perm)
    if N == 4:
        if (aperm[0],aperm[1]) in ((0,1),(1,0),(2,3),(3,2)): # N = 4
            translations.append(nk.utils.group._permutation_group.Permutation(aperm))
    if N == 16:
        if (aperm[0],aperm[1],aperm[3]) in ((0,1,3),(2,3,1),(8,9,11),(10,11,9)): # N = 16
    # if (aperm[0],aperm[1],aperm[2],aperm[3]) in ((4,7,6,5),): # N = 8
    # if (aperm[2],aperm[0]) in ((2,0),(3,0),(0,1),(1,2)):#,(2,3,1)): # N = 16, two dimers with just translation
    # if (aperm[0],aperm[1],aperm[3]) in ((0,1,3),(3,2,0),(2,3,1),(1,0,2)): # N = 16, two dimers plus second translation option
            translations.append(nk.utils.group._permutation_group.Permutation(aperm))
translation_group = nk.utils.group._permutation_group.PermutationGroup(translations,degree=SITES)
print("Out of", len(g.automorphisms()), "permutations,",len(translation_group), "translations were picked.")

Out of 64 permutations, 4 translations were picked.


## Hamoltonian definition
$$ H = J_{1} \sum\limits_{\langle i,j \rangle}^{L} \vec{\sigma}_{i} \cdot \vec{\sigma}_{j} + J_{2} \sum\limits_{\langle\langle i,j \rangle\rangle_{SS}}^{L}  \vec{\sigma}_{i} \cdot \vec{\sigma}_{j} + 2h\sum\limits_{i} \sigma^z_{i}\,. $$

Axiliary constant operators used to define hamiltonian are loaded from the external file, they are pre-defined in the `HamOps` class.

In [90]:
from lattice_and_ops import HamOps
ho = HamOps()
# H_Z = .5
ha_1 = nk.operator.GraphOperator(hilbert, graph=g, bond_ops=ho.bond_operator(JEXCH1,JEXCH2, h_z=H_Z, use_MSR=False), bond_ops_colors=ho.bond_color)
ha_2 = nk.operator.GraphOperator(hilbert, graph=g, bond_ops=ho.bond_operator(JEXCH1,JEXCH2, h_z=H_Z, use_MSR=True), bond_ops_colors=ho.bond_color)


### Magnetization operator definition
$$ \hat{m}_z := \sum\limits_i \sigma_i^z $$

In [91]:
m_z = sum(nk.operator.spin.sigmaz(hilbert, i) for i in range(hilbert.size))

In [92]:
if False: # exact diagonalization scan
    # H_Z = 0
    print("#N =", N)
    print("# J1     J2      H_Z            m_z            ground          1.excited          2.excited")
    for H_Z in np.arange(0.02,0.03,step=0.001):
        ha = nk.operator.GraphOperator(hilbert, graph=g, bond_ops=ho.bond_operator(JEXCH1,JEXCH2, h_z=H_Z, use_MSR=False), bond_ops_colors=ho.bond_color)
        start = time.time()
        evals, eigvects = nk.exact.lanczos_ed(ha, k=3, compute_eigenvectors=True)
        ground_state = eigvects.T[0]
        end = time.time()
        magnetization_z = ground_state.transpose()@(m_z@ground_state)
        # print("ED took", (start-end)/60, "min.")
        # print(np.round(np.array(ground_state),5))
        # print(ha.to_dense())
        print("%.3f   %.3f  %2.10f   %2.10f   %2.10f   %14.10f   %14.10f" % (JEXCH1, JEXCH2, H_Z, magnetization_z, evals[0], evals[1], evals[2]))

## Exact diagonalization

In [93]:
if g.n_nodes < 20:
    start = time.time()
    if g.n_nodes < 15:
        evals, eigvects = nk.exact.full_ed(ha_1, compute_eigenvectors=True)
    else:
        evals, eigvects = nk.exact.lanczos_ed(ha_1, k=3, compute_eigenvectors=True)
    end = time.time()
    diag_time = end - start
    print("Ground state energy:",evals[0], "\nIt took ", round(diag_time,2), "s =", round((diag_time)/60,2),"min")
else:
    print("System is too large for exact diagonalization. Setting exact_ground_energy = 0 (which is wrong)")
    evals = [0,0,0]
    eigvects = None 
exact_ground_energy = evals[0]

Ground state energy: -24.00000000000002 
It took  1.82 s = 0.03 min


# Machine learning starts here

## Machine definition and other auxiliary `netket` objects
We define two sets of these objects, usually: 
- variables ending with ...`_1` belongs to the choice of standard basis,
- variables ending with ...`_2` belongs to the choice of MSR basis.

But they can be used in a different way when we need to compare two different models.

In [95]:
# Selection of machine type
if MACHINE == "RBM":
    machine_1 = nk.models.RBM(dtype=DTYPE, alpha=ALPHA)#, use_visible_bias=False) 
    machine_2 = nk.models.RBM(dtype=DTYPE, alpha=ALPHA)#, use_visible_bias=False)
elif MACHINE == "RBMSymm":
    machine_1 = nk.models.RBMSymm(g.automorphisms(), dtype=DTYPE, alpha=ALPHA)#, use_visible_bias=False) 
    machine_2 = nk.models.RBMSymm(g.automorphisms(), dtype=DTYPE, alpha=ALPHA)#, use_visible_bias=False)
elif MACHINE == "RBMSymm_transl":
    machine_1 = nk.models.RBMSymm(translation_group, dtype=DTYPE, alpha=ALPHA)#, use_visible_bias=False) 
    machine_2 = nk.models.RBMSymm(translation_group, dtype=DTYPE, alpha=ALPHA)#, use_visible_bias=False)
elif MACHINE == "GCNN":
    machine_1 = nk.models.GCNN(symmetries=g.automorphisms(), dtype=DTYPE, 
        layers=N_LAYERS, features=FEATURES, characters=characters_dimer_1)#, output_activation=log_cosh)
    machine_2 = nk.models.GCNN(symmetries=g.automorphisms(), dtype=DTYPE, 
        layers=N_LAYERS, features=FEATURES, characters=characters_dimer_2)#, output_activation=log_cosh)
elif MACHINE == "myRBM":
    from GCNN_Nomura import GCNN_my
    machine_1 = GCNN_my(symmetries=g.automorphisms(), dtype=DTYPE, layers=1, features=ALPHA, characters=characters_dimer_1, output_activation=nk.nn.log_cosh, use_bias=True, use_visible_bias=True)
    machine_2 = GCNN_my(symmetries=g.automorphisms(), dtype=DTYPE, layers=1, features=ALPHA, characters=characters_dimer_2, output_activation=nk.nn.log_cosh, use_bias=True, use_visible_bias=True)
elif MACHINE == "Jastrow":
    from lattice_and_ops import Jastrow
    machine_1 = Jastrow()
    machine_2 = Jastrow()
elif MACHINE == "RBMModPhase":
    machine_1 = nk.models.RBMModPhase(alpha=ALPHA, use_hidden_bias=True, dtype=DTYPE)
    machine_2 = nk.models.RBMModPhase(alpha=ALPHA, use_hidden_bias=True, dtype=DTYPE)

    # A linear schedule varies the learning rate from 0 to 0.01 across 600 steps.
    modulus_schedule_1=optax.linear_schedule(0,0.01,NUM_ITER)
    modulus_schedule_2=optax.linear_schedule(0,0.01,NUM_ITER)
    # The phase starts with a larger learning rate and then is decreased.
    phase_schedule_1=optax.linear_schedule(0.05,0.01,NUM_ITER)
    phase_schedule_2=optax.linear_schedule(0.05,0.01,NUM_ITER)
    # Combine the linear schedule with SGD
    optm_1=optax.sgd(modulus_schedule_1)
    optp_1=optax.sgd(phase_schedule_1)
    optm_2=optax.sgd(modulus_schedule_2)
    optp_2=optax.sgd(phase_schedule_2)
    # The multi-transform optimizer uses different optimisers for different parts of the parameters.
    optimizer_1 = optax.multi_transform({'o1': optm_1, 'o2': optp_1}, flax.core.freeze({"Dense_0":"o1", "Dense_1":"o2"}))
    optimizer_2 = optax.multi_transform({'o1': optm_2, 'o2': optp_2}, flax.core.freeze({"Dense_0":"o1", "Dense_1":"o2"}))
else:
    raise Exception(str("undefined MACHINE: ")+str(MACHINE))

# Selection of sampler type
if SAMPLER == 'local':
    sampler_1 = nk.sampler.MetropolisLocal(hilbert=hilbert)
    sampler_2 = nk.sampler.MetropolisLocal(hilbert=hilbert)
elif SAMPLER == 'exact':
    sampler_1 = nk.sampler.ExactSampler(hilbert=hilbert)
    sampler_2 = nk.sampler.ExactSampler(hilbert=hilbert)
else:
    sampler_1 = nk.sampler.MetropolisExchange(hilbert=hilbert, graph=g)
    sampler_2 = nk.sampler.MetropolisExchange(hilbert=hilbert, graph=g)
    if SAMPLER != 'exchange':
        print("Warning! Undefined fq.SAMPLER:", SAMPLER, ", dafaulting to MetropolisExchange fq.SAMPLER")

if MACHINE != "RBMModPhase":
    optimizer_1 = nk.optimizer.Sgd(learning_rate=ETA)
    optimizer_2 = nk.optimizer.Sgd(learning_rate=ETA)

# Stochastic Reconfiguration as a preconditioner
sr_1  = nk.optimizer.SR(diag_shift=0.01)
sr_2  = nk.optimizer.SR(diag_shift=0.01)

# The variational state (former name: nk.variational.MCState)
vs_1 = nk.vqs.MCState(sampler_1 , machine_1 , n_samples=SAMPLES)
vs_2 = nk.vqs.MCState(sampler_2 , machine_2 , n_samples=SAMPLES)
vs_1.init_parameters(jax.nn.initializers.normal(stddev=0.01))
vs_2.init_parameters(jax.nn.initializers.normal(stddev=0.01))


gs_1 = nk.VMC(hamiltonian=ha_1 ,optimizer=optimizer_1 ,preconditioner=sr_1 ,variational_state=vs_1)
gs_2 = nk.VMC(hamiltonian=ha_2 ,optimizer=optimizer_2 ,preconditioner=sr_2 ,variational_state=vs_2) 

test


TypeError: Failed to hash Flax Module.  The module probably contains unhashable attributes.  Module=GCNN_Irrep(
    # attributes
    symmetries = HashableArray(wrapped=array([[ 0,  1,  2, ..., 13, 14, 15],
           [ 0,  1, 13, ...,  2, 14, 15],
           [ 0,  4,  7, ...,  8, 11, 15],
           ...,
           [15, 11,  8, ...,  7,  4,  0],
           [15, 14,  2, ..., 13,  1,  0],
           [15, 14, 13, ...,  2,  1,  0]]))
    irreps = (HashableArray(wrapped=array([[[1.]],
    
           [[1.]],
    
           [[1.]],
    
           [[1.]],
    
           [[1.]],
    
           [[1.]],
    
           [[1.]],
    
           [[1.]],
    
           [[1.]],
    
           [[1.]],
    
           [[1.]],
    
           [[1.]],
    
           [[1.]],
    
           [[1.]],
    
           [[1.]],
    
           [[1.]],
    
           [[1.]],
    
           [[1.]],
    
           [[1.]],
    
           [[1.]],
    
           [[1.]],
    
           [[1.]],
    
      

In [ ]:
if False:
    ETA2 = 0.1
    optimizer_1 = nk.optimizer.Sgd(learning_rate=ETA2)
    optimizer_2 = nk.optimizer.Sgd(learning_rate=ETA2)
    gs_1 = nk.VMC(hamiltonian=ha_1 ,optimizer=optimizer_1,preconditioner=sr_1,variational_state=vs_1)   # 0 ... symmetric
    gs_2 = nk.VMC(hamiltonian=ha_2 ,optimizer=optimizer_2,preconditioner=sr_2,variational_state=vs_2)   # 1 ... symmetric+MSR

In [ ]:
print(vs_1.n_parameters, vs_2.n_parameters)

16672 16672


# Calculation
We let the calculation run for `NUM_ITERS` iterations for both cases _1 and _2 (without MSR and with MSR). If only one case is desired, set the variable `no_of_runs` to 1.

In [ ]:
runs = [1,1]
no_of_runs = np.sum(runs) # 1 - one run for variables with ..._1;  2 - both runs for variables ..._1 and ..._2
run_only_2 = (runs[1]==1 and runs[0]==0) # in case of no_of_runs=1
NUM_ITER = 50#3000
print("J_1 =", JEXCH1,"; H_Z =",H_Z, end="; ")
if exact_ground_energy != 0:
    print("Expected exact energy:", exact_ground_energy)
for i,gs in enumerate([gs_1,gs_2][run_only_2:run_only_2+no_of_runs]):
    start = time.time()
    gs.run(out=OUT_NAME+str(i), n_iter=int(NUM_ITER))#, obs={'symmetry':P(0,1)})
    end = time.time()
    print("The calculation for {} of type {} took {} min".format(MACHINE, i+1, (end-start)/60))


J_1 = 0.2 ; H_Z = 0; Expected exact energy: -23.99999999999999


  2%|▏         | 1/50 [00:14<11:52, 14.54s/it, Energy=14.400011-0.000045j ± 0.000025 [σ²=0.000000, R̂=0.9980]]


KeyboardInterrupt: 

In [ ]:
for i,gs in enumerate([gs_1,gs_2][run_only_2:run_only_2+no_of_runs]):
    print("Trained RBM with MSR:" if (i+run_only_2) else "Trained RBM without MSR:")
    print("Energy =", gs.energy)
    print("m_z =", gs.estimate(m_z))

Trained RBM without MSR:
Energy = 119.56+0.74j ± 0.84 [σ²=501.04, R̂=1.0024]
m_z = 0.54+0.00j ± 0.30 [σ²=61.35, R̂=1.0021]
Trained RBM with MSR:
Energy = -118.59-0.02j ± 0.38 [σ²=99.92, R̂=0.9933]
m_z = -0.30+0.00j ± 0.19 [σ²=25.54, R̂=0.9977]


## Energy Convergence Plotting
In case that the machine did not converge, we can re-run the previous cell and than skip the next cell. This way, the replotting just appends the new results and does not erase the previoius results. 

In [ ]:
# Exact Energy Line
no_of_all_iters = NUM_ITER
figure = go.Figure(
    data=[
        go.Scatter(
            x=(0,no_of_all_iters),
            y=(exact_ground_energy,exact_ground_energy),
            mode="lines",line=go.scatter.Line(color="#000000",width=1), name="exact energy")],
    layout=go.Layout(
        template="simple_white",
        xaxis=dict(title="Iteration", mirror=True, showline=True),
        yaxis=dict(title="Energy", mirror=True, showline=True),
        title=("<b>"+"S-S"+" model </b>, L="+str(SITES)+", J2 ="+str(JEXCH2)+ ", J1 ="+str(JEXCH1)+" , η="+str(ETA)+", α="+str(ALPHA)+", samples="+str(SAMPLES)))
    ).add_hline(y=exact_ground_energy, opacity=1, line_width=1)


In [ ]:
# import the data from log file
OUT_NAME_suffixless=OUT_NAME
data = []
names = ["normal","MSR"]
for i in range(no_of_runs):
    data.append(json.load(open(OUT_NAME_suffixless+str(i)+".log")))
if type(data[0]["Energy"]["Mean"]) == dict: #DTYPE in (np.complex128, np.complex64):#, np.float64):# and False:
    energy_convergence = [data[i]["Energy"]["Mean"]["real"] for i in range(no_of_runs)]
    # symmetry = [data[i]["symmetry"]["Mean"]["real"] for i in range(no_of_runs-run_only_2)]
else:
    energy_convergence = [data[i]["Energy"]["Mean"] for i in range(no_of_runs)]
    # symmetry = [data[i]["symmetry"]["Mean"] for i in range(no_of_runs-run_only_2)]
for i in range(no_of_runs):
    figure.add_trace(go.Scatter(
        x=data[i]["Energy"]["iters"], y=energy_convergence[i],
        name=names[i]
    ))
    # figure.add_trace(go.Scatter(
    #     x=data[i]["Energy"]["iters"], y=symmetry[i],
    #     name=names[i]+"_swap"
    # ))

figure.update_layout(xaxis_title="Iteration",yaxis_title="Energy")
figure.show()

## Assessment of the other simulation results

In [ ]:
# Calculation of how long it took to reach 99.5% of exact energy. The first value under 0.5% deviation counts as a converged state.
no_of_runs = 2
threshold_energy = 0.995*exact_ground_energy
data = []
for i in range(no_of_runs):
    data.append(json.load(open(OUT_NAME+str(i)+".log")))
if type(data[0]["Energy"]["Mean"]) == dict:
    energy_convergence = [data[i]["Energy"]["Mean"]["real"] for i in range(no_of_runs)]
else:
    energy_convergence = [data[i]["Energy"]["Mean"] for i in range(no_of_runs)]
steps_until_convergence = [next((i for i,v in enumerate(energy_convergence[j]) if v < threshold_energy), -1) for j in range(no_of_runs)]
print(steps_until_convergence)

[-1, 0]


In [ ]:
# Evaluation of order parameters.
from lattice_and_ops import Operators, Lattice
ops = Operators(lattice,hilbert,ho.mszsz,ho.exchange)
for i,gs in enumerate([gs_1,gs_2][run_only_2:run_only_2+no_of_runs]):
    print("Trained RBM with MSR:" if (i+run_only_2) else "Trained RBM without MSR:")
    print("m_d^2 =", gs.estimate(ops.m_dimer_op))
    print("m_p(MSR) =", gs.estimate(ops.m_plaquette_op_MSR))
    print("m_s^2 =", gs.estimate(ops.m_s2_op))
    print("m_s^2(MSR) =", gs.estimate(ops.m_s2_op_MSR))

Trained RBM without MSR:
m_d^2 = -0.2716+0.0014j ± 0.0022 [σ²=0.0033, R̂=1.0085]
m_p(MSR) = 0.075-0.012j ± 0.054 [σ²=2.037, R̂=0.9933]
m_s^2 = 0.03790-0.00009j ± 0.00079 [σ²=0.00044, R̂=1.0058]
m_s^2(MSR) = 0.8781-0.0010j ± 0.0045 [σ²=0.0145, R̂=1.0125]
Trained RBM with MSR:
m_d^2 = -0.3015+0.0001j ± 0.0013 [σ²=0.0012, R̂=1.0041]
m_p(MSR) = 0.004-0.002j ± 0.052 [σ²=1.925, R̂=1.0018]
m_s^2 = 0.0430+0.0000j ± 0.0017 [σ²=0.0021, R̂=0.9988]
m_s^2(MSR) = 0.8806-0.0004j ± 0.0047 [σ²=0.0154, R̂=1.0004]


### Print final results

In [ ]:
from lattice_and_ops import log_results
log_results(JEXCH1,gs_1,gs_2,ops,SAMPLES,NUM_ITER,steps_until_convergence)

TypeError: log_results() missing 1 required positional argument: 'steps_until_convergence'

In [ ]:
print("{:6.3f} {:10.5f} {:8.5f}  {:7.4f} {:7.4f}  {:7.4f} {:7.4f}  {:7.4f} {:7.4f}  {:5.0f} {:5.0f}".format(JEXCH1, 
    gs_1.energy.mean.real,                          gs_1.energy.variance,
    gs_1.estimate(ops.m_dimer_op).mean.real,        gs_1.estimate(ops.m_dimer_op).variance,
    gs_1.estimate(ops.m_plaquette_op).mean.real,    gs_1.estimate(ops.m_plaquette_op).variance,
    gs_1.estimate(ops.m_s2_op).mean.real,           gs_1.estimate(ops.m_s2_op).variance, 
    SAMPLES, NUM_ITER, steps_until_convergence[0], sep='    '))

 0.400  -15.74737  0.46486   0.4879  0.0093   0.0233  0.4016   0.1384  0.0085   1000   400


In [ ]:
print("asd",file=None)

asd


In [ ]:
print("{:9.5f}".format(gs_1.energy.variance))

  0.46486


In [ ]:
gs_1.energy.to_dict()

{'Mean': (-15.747368841964612-0.003949807027622352j),
 'Variance': 0.46486127291880336,
 'Sigma': 0.02484698569889124,
 'R_hat': nan,
 'TauCorr': 0.16403971925269567}